# Inventory Detection System - Mobile Approach

This notebook demonstrates creating a TensorFlow Lite model for detecting and counting Pepsi and Kilimanjaro products on mobile devices.

## Overview

**Problem**: Manual counting of inventory takes hours each week and is often skipped, causing errors.

**Solution**: We'll develop a mobile-friendly model that can run directly on Android or iOS devices.

**Advantages**:
- On-device processing (no internet required)
- Real-time detection in the field
- Privacy-preserving (data stays on device)
- Works in areas with poor connectivity

## 1. Setup and Imports

In [ ]:
# Install required packages
!pip install tensorflow tensorflow-hub numpy pillow matplotlib

In [ ]:
import os
import json
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, FileLink

## 2. Check for GPU Availability

While not strictly necessary for this approach, GPU acceleration can speed up training.

In [ ]:
# Check if TensorFlow can see a GPU
if tf.config.list_physical_devices('GPU'):
    print(f"✅ GPU is available for TensorFlow")
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  - {gpu}")
else:
    print("⚠️ GPU is not available for TensorFlow. Using CPU instead. Training may be slower.")

## 3. View Sample Images

Let's first look at our sample images to understand what we're working with.

In [ ]:
# Display sample images
sample_dir = "data/sample_images"
sample_images = [f for f in os.listdir(sample_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

plt.figure(figsize=(15, 10))
for i, image_file in enumerate(sample_images):
    image_path = os.path.join(sample_dir, image_file)
    img = Image.open(image_path)
    
    plt.subplot(2, 2, i+1)
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 4. Prepare Data for Training

We'll organize our sample images into a structure suitable for training with Keras.

In [ ]:
def prepare_data(sample_dir, batch_size=32):
    """Prepare data for training."""
    print(f"Preparing data from {sample_dir}...")
    
    # Create a temporary directory for organizing data
    temp_dir = "temp_data"
    train_dir = os.path.join(temp_dir, "train")
    val_dir = os.path.join(temp_dir, "val")
    
    # Create class directories
    for class_name in ["pepsi", "kilimanjaro"]:
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    
    # Copy sample images to class directories
    for file_name in os.listdir(sample_dir):
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
            
        if "pepsi" in file_name.lower():
            class_name = "pepsi"
        elif "kilimanjaro" in file_name.lower():
            class_name = "kilimanjaro"
        else:
            print(f"Skipping unknown product: {file_name}")
            continue
        
        source_path = os.path.join(sample_dir, file_name)
        
        # Split files between train and val (80/20)
        if np.random.rand() < 0.8:
            dest_path = os.path.join(train_dir, class_name, file_name)
        else:
            dest_path = os.path.join(val_dir, class_name, file_name)
        
        shutil.copy(source_path, dest_path)
    
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Only rescaling for validation
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    print("✅ Data preparation completed!")
    
    # Display class mapping
    print(f"Class indices: {train_generator.class_indices}")
    
    return train_generator, validation_generator, train_generator.class_indices

In [ ]:
# Prepare data
batch_size = 8  # Small batch size since we have few samples
train_generator, validation_generator, class_indices = prepare_data(sample_dir, batch_size)

## 5. Create MobileNetV2 Model

MobileNetV2 is a lightweight model designed for mobile and edge devices.

In [ ]:
def create_model(input_shape=(224, 224, 3), num_classes=2):
    """Create a MobileNetV2 model for classification."""
    print("Creating MobileNetV2 model...")
    
    # Load base model
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add custom classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ MobileNetV2 model created successfully!")
    
    return model

# Create model
model = create_model(num_classes=len(class_indices))

# Display model summary
model.summary()

## 6. Train the Model

Now we'll train our model on the prepared data. Since we have very few samples, we'll use data augmentation and transfer learning to help prevent overfitting.

In [ ]:
def train_model(model, train_generator, validation_generator, epochs=10):
    """Train the model."""
    print(f"Training model for {epochs} epochs...")
    
    # Create a TensorBoard callback for visualization
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # Early stopping to prevent overfitting
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    # Train model
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[early_stop]
    )
    
    print("✅ Model training completed!")
    
    return model, history

In [ ]:
import datetime

# Train the model
epochs = 10
model, history = train_model(model, train_generator, validation_generator, epochs)

## 7. Visualize Training Results

In [ ]:
# Plot training history
def plot_training_history(history):
    """Plot training and validation metrics."""
    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Plot history
plot_training_history(history)

## 8. Save Keras Model

Let's save our trained model before converting it to TensorFlow Lite.

In [ ]:
# Create output directory
output_dir = "outputs/mobile"
os.makedirs(output_dir, exist_ok=True)

# Save Keras model
keras_model_path = os.path.join(output_dir, "model.h5")
model.save(keras_model_path)
print(f"✅ Keras model saved to {keras_model_path}")

## 9. Convert to TensorFlow Lite

Now we'll convert our Keras model to TensorFlow Lite format, which is optimized for mobile and embedded devices.

In [ ]:
def convert_to_tflite(model, output_path, quantize=True):
    """Convert model to TensorFlow Lite format."""
    print(f"Converting model to TensorFlow Lite format...")
    
    # Create TFLite converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Apply optimization if requested
    if quantize:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        print("Applying quantization to reduce model size...")
    
    # Convert model
    tflite_model = converter.convert()
    
    # Save model
    with open(output_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"✅ TensorFlow Lite model saved to {output_path}")
    print(f"  - Original model size: {os.path.getsize(keras_model_path) / (1024*1024):.2f} MB")
    print(f"  - TFLite model size: {os.path.getsize(output_path) / (1024*1024):.2f} MB")

# Convert to TFLite with quantization
tflite_model_path = os.path.join(output_dir, "model.tflite")
convert_to_tflite(model, tflite_model_path, quantize=True)

## 10. Create Model Metadata

To make the model easier to use on mobile devices, let's create a metadata file with important information.

In [ ]:
def create_model_metadata(class_indices, output_path):
    """Create metadata for the TFLite model."""
    # Invert class indices to get index-to-label mapping
    index_to_label = {v: k for k, v in class_indices.items()}
    
    # Create metadata
    metadata = {
        "model_type": "classification",
        "input_shape": [224, 224, 3],
        "labels": index_to_label,
        "preprocessing": {
            "rescale": "1./255",
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225]
        }
    }
    
    # Save metadata
    with open(output_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✅ Model metadata saved to {output_path}")

# Create metadata
metadata_path = os.path.join(output_dir, "metadata.json")
create_model_metadata(class_indices, metadata_path)

## 11. Test the TFLite Model

Let's test our TFLite model on an inventory image to make sure it works correctly.

In [ ]:
def load_tflite_model(model_path):
    """Load a TensorFlow Lite model."""
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    return interpreter

def preprocess_image(image_path, input_shape):
    """Preprocess an image for model input."""
    # Load and resize image
    image = Image.open(image_path).convert('RGB')
    image = image.resize((input_shape[1], input_shape[0]))
    
    # Convert to numpy array
    image_array = np.array(image, dtype=np.float32)
    
    # Normalize pixel values
    image_array = image_array / 255.0
    image_array = (image_array - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    
    # Add batch dimension
    image_array = np.expand_dims(image_array, axis=0)
    
    return image_array, image

def detect_product(interpreter, metadata, image_path):
    """Detect a product in an image using the TFLite model."""
    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Get input shape
    input_shape = input_details[0]['shape'][1:3]
    
    # Preprocess image
    input_data, original_image = preprocess_image(image_path, input_shape)
    
    # Display the image
    plt.figure(figsize=(8, 8))
    plt.imshow(original_image)
    plt.axis('off')
    plt.title('Input Image')
    plt.show()
    
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get predicted class
    predicted_class_index = np.argmax(output_data[0])
    # Load metadata
    with open(metadata, 'r') as f:
        metadata_dict = json.load(f)
    
    predicted_class = metadata_dict['labels'][str(predicted_class_index)]
    confidence = output_data[0][predicted_class_index]
    
    # Create result
    pepsi_count = 1 if predicted_class == "pepsi" else 0
    kilimanjaro_count = 1 if predicted_class == "kilimanjaro" else 0
    
    result = {
        "pepsi_count": pepsi_count,
        "kilimanjaro_count": kilimanjaro_count,
        "predicted_class": predicted_class,
        "confidence": float(confidence)
    }
    
    return result, original_image

def visualize_result(result, image):
    """Visualize detection result."""
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    
    # Add result as text
    plt.figtext(0.5, 0.01, f"Predicted: {result['predicted_class']} (Confidence: {result['confidence']:.2f})", 
               ha="center", fontsize=14, 
               bbox={"facecolor":"white", "alpha":0.8, "pad":5})
    
    plt.show()

In [ ]:
# Load TFLite model
interpreter = load_tflite_model(tflite_model_path)

# Test on sample images
for image_file in sample_images:
    image_path = os.path.join(sample_dir, image_file)
    result, image = detect_product(interpreter, metadata_path, image_path)
    
    print(f"Result for {image_file}:")
    print(f"  Predicted class: {result['predicted_class']}")
    print(f"  Confidence: {result['confidence']:.2f}")
    
    visualize_result(result, image)

## 12. A Note on Real-World Mobile Implementation

For a real mobile app, we would need to:

1. **Implement object detection**: The current model only classifies whole images. For actual shelf images, we would need either:
   - Object detection model (like YOLO, but converted to TFLite)
   - Sliding window approach with our classifier

2. **Create Android/iOS application**: Build a mobile app that:
   - Takes photos or loads images from gallery
   - Runs the TFLite model on these images
   - Displays results with counts and visualizations

Let's create a pseudocode example of how a mobile implementation would work:

```java
// Android pseudocode for product detection
public class ProductDetector {
    private Interpreter tfliteInterpreter;
    private Map<Integer, String> labelMap;
    
    public ProductDetector(Context context) {
        // Load model from assets
        tfliteInterpreter = new Interpreter(loadModelFile(context, "model.tflite"));
        
        // Load metadata
        labelMap = loadMetadata(context, "metadata.json");
    }
    
    public ProductResult analyzeImage(Bitmap image) {
        // Preprocess image
        ByteBuffer inputBuffer = preprocess(image);
        
        // Prepare output buffer
        float[][] outputBuffer = new float[1][2];  // 2 classes
        
        // Run inference
        tfliteInterpreter.run(inputBuffer, outputBuffer);
        
        // Process results
        int pepsiCount = 0;
        int kilimanjaroCount = 0;
        
        // In a real app, we would run object detection and count instances
        // Here we're just doing basic classification
        float[] scores = outputBuffer[0];
        int classIndex = argmax(scores);
        String className = labelMap.get(classIndex);
        
        if ("pepsi".equals(className)) {
            pepsiCount = 1;
        } else if ("kilimanjaro".equals(className)) {
            kilimanjaroCount = 1;
        }
        
        return new ProductResult(pepsiCount, kilimanjaroCount, className, scores[classIndex]);
    }
    
    // Helper methods would be implemented here
}
```

## 13. Mobile App Framework

Finally, let's generate a skeleton for the Android app that would use our TFLite model.

In [ ]:
def create_android_project(model_path, metadata_path, output_dir):
    """Create a basic Android project structure."""
    print(f"Creating Android project structure in {output_dir}...")
    
    # Create project directories
    project_dir = os.path.join(output_dir, "InventoryDetector")
    assets_dir = os.path.join(project_dir, "app/src/main/assets")
    java_dir = os.path.join(project_dir, "app/src/main/java/com/example/inventorydetector")
    
    os.makedirs(assets_dir, exist_ok=True)
    os.makedirs(java_dir, exist_ok=True)
    
    # Copy model and metadata to assets directory
    shutil.copy(model_path, os.path.join(assets_dir, "model.tflite"))
    shutil.copy(metadata_path, os.path.join(assets_dir, "metadata.json"))
    
    # Write Java files and resources (code would be too long to include here)
    # We would write the MainActivity.java, ModelExecutor.java, etc.
    
    print(f"✅ Android project structure created at {project_dir}")
    print("To build the Android app, you would need to open this project in Android Studio.")
    
    return project_dir

# Create Android project structure
android_dir = os.path.join(output_dir, "android")
project_dir = create_android_project(tflite_model_path, metadata_path, android_dir)

## 14. Conclusion

### Advantages of the Mobile Approach:
- **On-device processing**: Works without internet connection
- **Privacy-preserving**: Data stays on the device
- **Real-time performance**: Optimized for mobile hardware
- **Accessibility**: Works in remote locations with poor connectivity

### Limitations:
- **Limited model size**: Mobile devices have constraints on model complexity
- **Reduced accuracy**: Quantized models may have slightly lower accuracy
- **Development complexity**: Requires mobile app development skills
